In [31]:
# Dependencies
from bs4 import BeautifulSoup
import requests,time
from splinter import Browser
import pandas as pd
import pymongo
import datetime as dt

### NASA Mars News###

In [32]:
# @NOTE: Replace the path with your actual path to the chromedriver
executable_path = {"executable_path": "C:/Users/Keke/.wdm/drivers/chromedriver/win32/84.0.4147.30/chromedriver.exe"}
browser= Browser("chrome", **executable_path, headless=False)

url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [33]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [34]:
# scrape newest title
news_title = soup.find_all("div", class_="content_title")[1].text
print(news_title)



Alabama High School Student Names NASA's Mars Helicopter




In [35]:
# scrape newest paragraph
news_p= soup.find_all("div", class_="article_teaser_body")[1].text
print(news_p)

IndexError: list index out of range

### JPL Mars Space Images - Featured Image ###

In [36]:
# Vist JPL Page
executable_path = {"executable_path": "C:/Users/Keke/.wdm/drivers/chromedriver/win32/84.0.4147.30/chromedriver.exe"}
browser= Browser("chrome", **executable_path, headless=False)

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [37]:
# Navigating site by ask Splinter to Go to Site and Click Button with Class Name full_image
# <button class="full_image">Full Image</button>
image= browser.find_by_id("full_image")
image.click()

In [38]:
# Find "More Info" Button and Click It
browser.is_element_present_by_text("more info", wait_time=1)
more_info= browser.links.find_by_partial_text("more info")
more_info.click()

In [39]:
# Parse Results HTML with BeautifulSoup
html = browser.html
imagesoup = BeautifulSoup(html, "html.parser")

In [40]:
# Based on inspect html-  <figure class="lede">
# 			  <a href="/spaceimages/images/largesize/PIA18614_hires.jpg"><img alt="This vista from NASA's Mars Exploration Rover Opportunity shows 'Wdowiak Ridge,' from left foreground to center, as part of a northward look with the rover's tracks visible at right." title="This vista from NASA's Mars Exploration Rover Opportunity shows 'Wdowiak Ridge,' from left foreground to center, as part of a northward look with the rover's tracks visible at right." class="main_image" src="/spaceimages/images/largesize/PIA18614_hires.jpg"></a>
# 			</figure>
featured_img_url = imagesoup.select_one("figure.lede a img").get("src")
featured_img_url

'/spaceimages/images/largesize/PIA22893_hires.jpg'

In [41]:
# Create unique URL
featured_img_url = f"https://www.jpl.nasa.gov{featured_img_url}"
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22893_hires.jpg


### Mars Facts ###

In [48]:
# Visit the Mars Facts Site Using Pandas to Read
df = pd.read_html("https://space-facts.com/mars/")[0]
print(df)
df.columns=["Description", "Value"]
df.set_index("Description", inplace=True)
df

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres ###

In [49]:
# Visit the USGS Astrogeology site
executable_path = {"executable_path": "C:/Users/Keke/.wdm/drivers/chromedriver/win32/84.0.4147.30/chromedriver.exe"}
browser= Browser("chrome", **executable_path, headless=False)

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [50]:
hemisphere_image_urls = []

itemlink= browser.find_by_css("a.product-item h3")

for item in range(len(itemlink)):
    
    hemisphere= {}
    #click on to each h3 link
    browser.find_by_css("a.product-item h3")[item].click()
    
    # find sample image tag & extract <href>
    sample = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample["href"]
    
    # retrieve hemisphere title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # append to List
    hemisphere_image_urls.append(hemisphere)
    
    # navigate backwards
    browser.back()
    

hemisphere_image_urls

C:\Users\Keke\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [51]:
#scrape all 
executable_path = {"executable_path": "C:/Users/Keke/.wdm/drivers/chromedriver/win32/84.0.4147.30/chromedriver.exe"}
browser= Browser("chrome", **executable_path, headless=False)

In [54]:
def mars_news, mars_images, mars_facts, mars_hem:
    news_title, news_p = mars_news(browser)
    featured_img_url= mars_images(browser)
    facts = mars_facts(browser)
    hemisphere_image_urls = mars_hem (browser)
    timestamp = dt.datetime.now()


    mars_data= {
        "news_title": news_title,
        "news_paragraph": news_p,
        "mars_image": featured_img_url,
        "mars_facts": facts,
        "mars_hem": hemisphere_image_urls,
        "latest":timestamp
    }

return mars_data

SyntaxError: invalid syntax (<ipython-input-54-46edccd445a1>, line 1)